In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('./amazon_reviews.csv')


In [3]:
# Loại bỏ các cột không cần thiết
df = df.drop(columns=['verified', 'image', 'reviewTime', 'summary', 'reviewText', 'vote'])

In [4]:

# Thay thế chuỗi '[]' bằng chuỗi rỗng ''
df['feature'] = df['feature'].replace('[]', '')
# Loại bỏ các dòng có giá trị rỗng sau khi thay thế
df = df[df['feature'] != '']

# Loại bỏ các dòng có giá trị NaN trong cột 'price'
df = df.dropna(subset=['price'])
df.head()

,userName,itemName,description,brand,feature,category,price,rating
2,Jaclyn,"Pet Champion Adjustable No-Pull Harness, Colla...","[""The Pet Champion Large/ Extra Large 22-36 in...",Pet Champion,"['Features Bright Pink Lemonade pattern', 'Dua...",Pet_Supplies,$7.99,5.0
3,Christinne M,Koh-I-Noor Progresso Woodless Colored 24-Penci...,['Koh-I-Noor Progresso Woodless Colored 24-Pen...,KOH-I-NOOR,"['Rich, pigmented, solid color encased in lacq...",Arts_Crafts_and_Sewing,$14.18,5.0
6,patrick,Blue Buffalo Life Protection Formula Natural P...,['Because puppyhood is such an important stage...,Blue Buffalo,['HIGH QUALITY PUPPY FOOD: Blue Buffalo always...,Pet_Supplies,$28.99,5.0
7,Craig S.,Squishy Face Studio Flirt Pole V2 with Braided...,"[""Want An Easy Way To Keep Your Dog Happy And ...",Squishy Face Studio,['New and improved V2 developed using customer...,Pet_Supplies,$31.38,3.0
8,Super Sanne,Penn Plax Net Breeder for Aquarium,['The Net Breeder by Penn Plax is the safe way...,Penn Plax,['ISOLATE THE FRY: the Net Breeder Deluxe is t...,Pet_Supplies,$5.49,2.0


In [5]:
# Bộ dữ liệu khá lớn, tuy nhiên lại có một số tên người dùng lại không có (Amazon Customer)
# Do cần đánh giá rate nên lọc một số userName mà rate < 20, itemName < 5
# Đếm số lần xuất hiện của mỗi userName và lọc
user_counts = df['userName'].value_counts()
valid_usernames = user_counts[user_counts >= 20].index

# Đếm số lần xuất hiện của mỗi itemName và lọc
item_counts = df['itemName'].value_counts()
valid_items = item_counts[item_counts >= 5].index

# Lọc dữ liệu dựa trên userName và itemName
df_filtered = df[~df['userName'].isin(['Amazon Customer']) & 
                 df['userName'].isin(valid_usernames) &
                 df['itemName'].isin(valid_items)]

In [6]:
#shape of the dataframe
df_filtered.shape

(48098, 8)

In [7]:
df = df_filtered.copy()

In [8]:
# Lấy danh sách duy nhất các userName
users = df[['userName']].drop_duplicates().reset_index(drop=True)
# Tạo userId tự tạo
users['userId'] = users.index + 1  # Bắt đầu từ 1

# Chọn cột cần thiết
users = users[['userId', 'userName']]

# Lưu vào tệp CSV
users.to_csv('users.csv', index=False)


In [9]:
# Lấy danh sách duy nhất các itemName và feature
items = df[['itemName', 'feature', 'price']].drop_duplicates().reset_index(drop=True)

# Tạo itemId tự tạo
items['itemId'] = items.index + 1  # Bắt đầu từ 1

# Chọn cột cần thiết
items = items[['itemId', 'itemName', 'feature', 'price']]

# Lưu vào tệp CSV
items.to_csv('items.csv', index=False)


In [10]:

# Đọc lại tệp users và items
users = pd.read_csv('users.csv')
items = pd.read_csv('items.csv')

# Merge để lấy userId và itemId
ratings = df.merge(users, on='userName', how='left')
ratings = ratings.merge(items, on='itemName', how='left')

# Chọn các cột cần thiết
ratings = ratings[['userId', 'itemId', 'rating']]

# Sắp xếp dữ liệu theo cột 'userId' tăng dần
ratings_sorted = ratings.sort_values(by='userId', ascending=True)

# Kiểm tra dữ liệu sau khi sắp xếp
print("\nDữ liệu :")
print(ratings_sorted.head())

# Lưu dữ liệuvào tệp 'ratings.csv'
ratings_sorted.to_csv('ratings.csv', index=False)



Dữ liệu :
       userId  itemId  rating
0           1       1     5.0
42344       1     778     5.0
42345       1    1365     5.0
42346       1    1559     5.0
42420       1     778     5.0
